In [1]:
using Revise
includet("../cluster_env/runs/rg_opt_unstructured/job.jl")

In [2]:
using Base.Threads, OhMyThreads
using BenchmarkTools
using JLD2
using Geppetto
using Random, Distributions, StatsBase
using Logging

using Optimization, OptimizationBBO

In [4]:
using CairoMakie
using GLMakie
CairoMakie.activate!()
# display(GLMakie.Screen(), f.figure)

# Version 1 (testing, N fixed)

In [5]:
s, t = do_opt([0.9, 2., 1., 0.2, 0.5, 0.2];
    solver=OptimizationBBO.BBO_separable_nes(),
    maxtime=20,
    timelimit=5
)
s.objective

-0.07 - [1.7506365021860937, 2.764349576832359, 0.8867609274570353, 0.15302233864055, 0.8763735051443757, 0.17897633313623365]
-0.07 - [1.7506365021860937, 2.764349576832359, 0.8867609274570353, 0.15302233864055, 0.8763735051443757, 0.17897633313623365]
-0.17 - [0.5891824484618873, 2.6012381345241127, 0.9904206344258687, 0.12045884586538046, 0.6334459676085582, 0.48161924457171645]
 33.190376 seconds (49.02 M allocations: 6.313 GiB, 1.44% gc time, 398.02% compilation time: <1% of which was recompilation)


-0.17

In [35]:
s, t = do_opt([0.9, 2., 1., 0.2, 0.5, 0.2];
    solver=Optim.ParticleSwarm(),
    maxtime=60,
    timelimit=5,
    opt_bounds=false
)
s.objective

-0.02 - [0.9, 2.0, 1.0, 0.2, 0.5, 0.2]
-0.02 - [0.9, 2.0, 1.0, 0.2, 0.5, 0.2]
-0.02 - [0.9, 2.0, 1.0, 0.2, 0.5, 0.2]
-0.03 - [0.9, 2.0, 1.0, 0.3360077364410511, 0.5, 0.2]
-0.06 - [0.7232334331426945, 2.681927176171312, 0.9165639554694722, 0.5369932067720602, 0.6519849902595407, 0.1612941320750216]
-0.09 - [0.997187913129125, 2.925592389441608, 0.9646558893890944, 0.787371910885112, 0.829252749006762, 0.14898784214959956]
-0.11 - [0.367453236097854, 5.511105841531281, 0.9401612767035017, 0.684288277077467, 0.6802234593269944, 0.26917528483275743]
-0.11 - [0.367453236097854, 5.511105841531281, 0.9401612767035017, 0.684288277077467, 0.6802234593269944, 0.26917528483275743]
-0.11 - [0.367453236097854, 5.511105841531281, 0.9401612767035017, 0.684288277077467, 0.6802234593269944, 0.26917528483275743]
-0.13 - [0.39454441894642256, 3.744700764643164, 0.9394634534198767, 0.9172202512770082, 0.736594817963572, 0.2745014625112158]
-0.13 - [0.39454441894642256, 3.744700764643164, 0.939463453419876

-0.13

In [74]:
s, t = do_opt([0.9, 2., 1., 0.2, 0.5, 0.2];
    solver=OptimizationBBO.BBO_generating_set_search(),
    maxtime=60*5,
    timelimit=5
)
s.objective

┌ Warning: Getting less than 90.0% good runs!! cm is Dict(-1007 => 100)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:271
┌ Warning: Getting less than 90.0% good runs!! cm is Dict(-1007 => 100)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:271


-0.03 - [0.9, 2.5, 1.0, 0.2, 0.5, 0.2]
-0.04 - [0.9, 3.5, 1.0, 0.2, 0.5, 0.2]
-0.07 - [0.897783286293195, 3.5, 1.0, 0.2, 0.5, 0.2]
-0.08 - [0.22094934693785578, 3.5, 1.0, 0.2, 0.5, 0.2]
-0.25 - [0.22094934693785578, 3.5, 1.0, 0.2, 0.6643541985029496, 0.2]
-0.25 - [0.22094934693785578, 3.5, 1.0, 0.2, 0.6643541985029496, 0.2]
-0.25 - [0.22094934693785578, 3.5, 1.0, 0.2, 0.6643541985029496, 0.2]
315.755481 seconds (111.91 M allocations: 17.748 GiB, 0.94% gc time)


-0.25

┌ Error: Error watching manifest
│   exception =
│    cannot set type for global OpenSSL_jll.best_wrapper. It already has a value or is already set to a different type.
│    Stacktrace:
│     [1] top-level scope
│       @ ~/.julia/packages/JLLWrappers/GfYNv/src/toplevel_generators.jl:121
│    Revise evaluation error at /home/honza/.julia/packages/JLLWrappers/GfYNv/src/toplevel_generators.jl:121
│    
│    Stacktrace:
│     [1] methods_by_execution!(interp::JuliaInterpreter.NonRecursiveInterpreter, methodinfo::Revise.CodeTrackingMethodInfo, mod::Module, ex::Expr; mode::Symbol, disablebp::Bool, always_rethrow::Bool, kwargs::@Kwargs{})
│       @ Revise ~/.julia/packages/Revise/7QZSP/src/lowered.jl:294
└ @ Revise ~/.julia/packages/Revise/7QZSP/src/pkgs.jl:473


In [207]:
do_run1(s.u...; num_repeats=1000)

0.169

# BOSS

In [9]:
using BOSS

In [10]:
function blackbox(x; noise_std=0.1)
    y = exp(x[1]/10) * cos(2*x[1])
    z = (1/2)^6 * (x[1]^2 - (15.)^2)
    
    y += rand(Normal(0., noise_std))
    z += rand(Normal(0., noise_std))

    return [y,z]
end

blackbox (generic function with 1 method)

# Bayesian Optimization from scikit-optimize

In [20]:
using PythonCall
np = pyimport("numpy")
skopt = pyimport("skopt")
skoplots = pyimport("skopt.plots")
plt = pyimport("matplotlib.pyplot")

Python: <module 'matplotlib.pyplot' from '/home/honza/UCLOneDrive/Projects/SSMC/.CondaPkg/.pixi/envs/default/lib/python3.12/site-packages/matplotlib/pyplot.py'>

In [21]:
pc_func = u -> -do_run1(u...;
    num_repeats=1000,
    timelimit=5,
)
maxreal = 10.
@time r = skopt.gp_minimize(pc_func,
    [(0., maxreal), (0., maxreal), (0., 1.), (0., 1.), (0., 1.), (0., 1.)];
    n_calls=500
);

┌ Warning: Getting less than 90.0% good runs!! cm is Dict(-1015 => 306, -2000 => 1, -1004 => 693)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:280
┌ Warning: Getting less than 90.0% good runs!! cm is Dict(11 => 1000)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:280
┌ Warning: Getting less than 90.0% good runs!! cm is Dict(-1015 => 306, -2000 => 1, -1004 => 693)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:280
┌ Warning: Getting less than 90.0% good runs!! cm is Dict(11 => 1000)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:280
┌ Warning: Getting less than 90.0% good runs!! cm is Dict(-1015 => 45, -1004 => 955)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:280
┌ Warning: Getting less than 90.0% good runs!! cm is Dict(-1015 => 45, -1004 => 955)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstruct

9304.914564 seconds (7.39 G allocations: 1.068 TiB, 0.93% gc time)


┌ Warning: Getting less than 90.0% good runs!! cm is Dict(-1015 => 266, -1004 => 734)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:280


In [27]:
r

Python:
          fun: -0.22991967871485944
            x: [0.32552484621545463, 0.797225758288552, 1.0, 0.1154682260716453…
    func_vals: [-0.000e+00 -0.000e+00 ... -5.005e-03 -2.004e-03]
      x_iters: [[0.7884550941442593, 4.7176914856543215, 0.10996939316703261, 0…
       models: [GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, …
                                        n_restarts_optimizer=2, noise='gaussian…
                                        normalize_y=True, random_state=19906879…
                                        n_restarts_optimizer=2, noise='gaussian…
                                        normalize_y=True, random_state=19906879…
                                        n_restarts_optimizer=2, noise='gaussian…
                                        normalize_y=True, random_state=19906879…
                                        n_restarts_optimizer=2, noise='gaussian…
                                        normalize_y=True, random_state=19906879…


In [67]:
skoplots.plot_convergence(r)
plt.show()

Python: None

In [96]:
skoplots.plot_objective(r;
    dimensions=["m", "c", "l", "si", "sr", "sb"]
)
plt.show()

Python: None

In [89]:
?skoplots.plot_objective

Python function `plot_objective`.

```text
Plot a 2-d matrix with so-called Partial Dependence plots of the objective
function. This shows the influence of each search-space dimension on the objective
function.

This uses the last fitted model for estimating the objective function.

The diagonal shows the effect of a single dimension on the
objective function, while the plots below the diagonal show
the effect on the objective function when varying two dimensions.

The Partial Dependence is calculated by averaging the objective value
for a number of random samples in the search-space,
while keeping one or two dimensions fixed at regular intervals. This
averages out the effect of varying the other dimensions and shows
the influence of one or two dimensions on the objective function.

Also shown are small black dots for the points that were sampled
during optimization.

A red star indicates per default the best observed minimum, but
this can be changed by changing argument ´minimum´.

.. note::
      The Partial Dependence plot is only an estimation of the surrogate
      model which in turn is only an estimation of the true objective
      function that has been optimized. This means the plots show
      an "estimate of an estimate" and may therefore be quite imprecise,
      especially if few samples have been collected during the
      optimization
      (e.g. less than 100-200 samples), and in regions of the search-space
      that have been sparsely sampled (e.g. regions away from the optimum).
      This means that the plots may change each time you run the
      optimization and they should not be considered completely reliable.
      These compromises are necessary because we cannot evaluate the
      expensive objective function in order to plot it, so we have to use
      the cheaper surrogate model to plot its contour. And in order to
      show search-spaces with 3 dimensions or more in a 2-dimensional
      plot,
      we further need to map those dimensions to only 2-dimensions using
      the Partial Dependence, which also causes distortions in the plots.

Parameters
----------
result : `OptimizeResult`
    The optimization results from calling e.g. `gp_minimize()`.

levels : int, default=10
    Number of levels to draw on the contour plot, passed directly
    to `plt.contourf()`.

n_points : int, default=40
    Number of points at which to evaluate the partial dependence
    along each dimension.

n_samples : int, default=250
    Number of samples to use for averaging the model function
    at each of the `n_points` when `sample_method` is set to 'random'.

size : float, default=2
    Height (in inches) of each facet. Ignored if ``ax`` is provided.

wspace : float, default=0.35
    The width of the padding between subplots, as a fraction of the
    average Axes width. Ignored if ``ax`` is provided.

hspace : float, default=0.35
   The height of the padding between subplots, as a fraction of the
   average Axes height. Ignored if ``ax`` is provided.

zscale : str, default='linear'
    Scale to use for the z axis of the contour plots. Either 'linear'
    or 'log'.

dimensions : list of str, default=None
    Labels of the dimension
    variables. `None` defaults to `space.dimensions[i].name`, or
    if also `None` to `['X_0', 'X_1', ..]`.

plot_dims : list of str and int, default=None
    List of dimension names or dimension indices from the
    search-space dimensions to be included in the plot.
    If `None` then use all dimensions except constant ones
    from the search-space.

sample_source : str or list of floats, default='random'
    Defines to samples generation to use for averaging the model function
    at each of the `n_points`.

    A partial dependence plot is only generated, when `sample_source`
    is set to 'random' and `n_samples` is sufficient.

    `sample_source` can also be a list of
    floats, which is then used for averaging.

    Valid strings:

    - 'random' - `n_samples` random samples will used
    - 'result' - Use only the best observed parameters
    - 'expected_minimum' - Parameters that gives the best
      minimum Calculated using scipy's minimize method.
      This method currently does not work with categorical values.
    - 'expected_minimum_random' - Parameters that gives the
      best minimum when using naive random sampling.
      Works with categorical values.

minimum : str or list of floats, default = 'result'
    Defines the values for the red points in the plots.
    Valid strings:

    - 'result' - Use best observed parameters
    - 'expected_minimum' - Parameters that gives the best
      minimum Calculated using scipy's minimize method.
      This method currently does not work with categorical values.
    - 'expected_minimum_random' - Parameters that gives the
      best minimum when using naive random sampling.
      Works with categorical values

n_minimum_search : int, default = None
    Determines how many points should be evaluated
    to find the minimum when using 'expected_minimum' or
    'expected_minimum_random'. Parameter is used when
    `sample_source` and/or `minimum` is set to
    'expected_minimum' or 'expected_minimum_random'.

show_points: bool, default = True
    Choose whether to show evaluated points in the
    contour plots.

cmap: str or Colormap, default = 'viridis_r'
    Color map for contour plots. Passed directly to
    `plt.contourf()`

ax: `Matplotlib.Axes`, default= None
    An axis object in which to plot the dependence plot. If provided,
    ``size`` is ignored and the caller is responsible for the size of the
    plot.

Returns
-------
ax : `Matplotlib.Axes`
    The axes object the plot was drawn in
```


In [66]:
jldsave("../data/rg_opt_scikit_bo.jld2";
    r,
    us=[pyconvert(Vector, x) for x in r.x_iters],
    fvals=pyconvert(Vector, r.func_vals)
)

# Testing noisy optimization

In [44]:
using OptimizationOptimJL, OptimizationBBO, OptimizationNOMAD

In [38]:
function tfunc1(m, s=1, N=5)
    d = Normal(m, s)
    sleep(0.1)
    mean(rand(d, N))
end

tfunc1 (generic function with 3 methods)

In [39]:
opf = OptimizationFunction((u,p)->tfunc1(u[1]))
opp = OptimizationProblem(opf, [-2.]; lb=[-10.], ub=[1000])
# opp = OptimizationProblem(opf, [-2.])

OptimizationProblem. In-place: true
u0: 1-element Vector{Float64}:
 -2.0

In [42]:
@time s = solve(opp, OptimizationBBO.BBO_generating_set_search(); maxtime=1)

  1.314570 seconds (316 allocations: 16.781 KiB)


retcode: MaxTime
u: 1-element Vector{Float64}:
 -9.996961195627406

In [43]:
s.original.f_calls

12

# noisyopt

In [12]:
using PythonCall
no = pyimport("noisyopt")
# np = pyimport("numpy")
# skopt = pyimport("skopt")
# skoplots = pyimport("skopt.plots")
# plt = pyimport("matplotlib.pyplot")

Python: <module 'noisyopt' from '/home/honza/UCLOneDrive/Projects/SSMC/.CondaPkg/.pixi/envs/default/lib/python3.12/site-packages/noisyopt/__init__.py'>

In [18]:
function pc_func2(u)
    -do_run1(u...;
        num_repeats=100,
        timelimit=5,
    )
end
maxreal = 10.
@time r = no.minimizeSPSA(pc_func2, [0.9, 2., 1., 0.2, 0.5, 0.2];
    # niter=3,
    # bounds=[(0., maxreal), (0., maxreal), (0., 1.), (0., 1.), (0., 1.), (0., 1.)],
    # bounds=[fill(0., 6), [maxreal, maxreal, 1., 1., 1., 1.]],
    # feps=0.1,
    paired=false,
    disp=true,
);

┌ Warning: getting invalid params - (1.9, 1.0, 0.0, -0.8, 1.5, 1.2)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:237
┌ Warning: getting invalid params - (-0.09999999999999998, 3.0, 2.0, 1.2, -0.5, -0.8)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:237
┌ Warning: getting invalid params - (-0.032386486436832396, 2.9323864864368323, 0.06761351356316758, 1.1323864864368325, 1.4323864864368323, 1.1323864864368325)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:237
┌ Warning: getting invalid params - (1.8323864864368324, 1.0676135135631677, 1.9323864864368323, -0.7323864864368324, -0.4323864864368324, -0.7323864864368324)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:237
┌ Warning: getting invalid params - (1.7949746893567633, 2.894974689356763, 0.10502531064323672, 1.0949746893567633, 1.3949746893567632, 1.0949746893567633)
└ @ Main ~/UCLOneDrive/Proje

  1.235131 seconds (1.96 M allocations: 318.412 MiB)


┌ Warning: getting invalid params - (1.537698883222034, 1.3623011167779662, 1.6376988832220338, -0.4376988832220338, -0.13769888322203383, 0.8376988832220338)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:237
┌ Warning: getting invalid params - (0.2623011167779662, 2.6376988832220336, 0.36230111677796617, 0.8376988832220338, 1.1376988832220338, -0.4376988832220338)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:237
┌ Warning: getting invalid params - (1.5369547130921644, 2.6369547130921642, 1.6369547130921642, -0.4369547130921643, -0.13695471309216434, -0.4369547130921643)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:237
┌ Warning: getting invalid params - (0.2630452869078357, 1.3630452869078358, 0.36304528690783566, 0.8369547130921644, 1.1369547130921642, 0.8369547130921644)
└ @ Main ~/UCLOneDrive/Projects/SSMC/cluster_env/runs/rg_opt_unstructured/job.jl:237
┌ Warning: getting inva

# NOMAD?

# Extra

In [189]:
GC.gc()
empty!(Out)
GC.gc()